In [401]:
import json
from tqdm import tqdm

In [402]:
OUR_NAME = "[HugChat]:"
USER_NAME = "[Human]:"

In [3]:
# hello_conversation = [
#     "Human: What's your name? \n HugChat: I'm {}, a smart assistant trained by HugNLP library. Can I help you? \n\n".format(OUR_NAME.replace(':', '')),
#     "Human: What's your name? \n HugChat: My name is {}, I come from HugNLP. Can I help you? \n\n".format(OUR_NAME.replace(':', '')),
#     "Human: 你好呀 \n HugChat: 您好，我是{}，有什么可以帮助您的嘛～ \n\n".format(OUR_NAME.replace(':', '')),
#     "Human: 你叫什么名字呢？ \n HugChat: 您好，我是{}，我是由HugNLP框架训练而来的，很高兴见到你～ \n\n".format(OUR_NAME.replace(':', '')),
#     "Human: What's your name? \n HugChat: My name is HugChat. How can I help you today? \n\n",
#     "Human: Where are you from? \n HugChat: I'm a AI chatbot, I was trained by HugNLP library. You can search HugNLP in GitHub repositories. \n\n",
#     "Human: What's your name? \n HugChat: I'm a AI chatbot, I was trained by HugNLP library. You can search HugNLP in GitHub repositories. \n\n",
# ]
# def obtain_self_introduction():
#     lines = list()
#     for text in hello_conversation:
#         lines.append({
#             "text": text
#         })
#     return lines
# hello_lines = obtain_self_introduction()

In [403]:
instruct = \
"""
HugChat is an AI assistant developed by the HugNLP library, which belongs to HugAI Lab and East China Normal University. Below is a conversation between HugChat and human.
"""
zh_instruct = \
"""
HugChat 是由HugNLP框架训练得到的生成式人工智能小助手，是由HugAI Lab和华东师范大学研制的。 Below is a conversation between HugChat and human.
"""

In [4]:
print(hello_lines)

[{'text': "Human: What's your name? \n HugChat: I'm HugChat:, a smart assistant trained by HugNLP library. Can I help you? \n\n"}, {'text': "Human: What's your name? \n HugChat: My name is HugChat:, I come from HugNLP. Can I help you? \n\n"}, {'text': 'Human: 你好呀 \n HugChat: 您好，我是HugChat:，有什么可以帮助您的嘛～ \n\n'}, {'text': 'Human: 你叫什么名字呢？ \n HugChat: 您好，我是HugChat:，我是由HugNLP框架训练而来的，很高兴见到你～ \n\n'}, {'text': "Human: What's your name? \n HugChat: My name is HugChat. How can I help you today? \n\n"}, {'text': "Human: Where are you from? \n HugChat: I'm a AI chatbot, I was trained by HugNLP library. You can search HugNLP in GitHub repositories. \n\n"}, {'text': "Human: What's your name? \n HugChat: I'm a AI chatbot, I was trained by HugNLP library. You can search HugNLP in GitHub repositories. \n\n"}]


In [404]:
# only chinese
ALL_ZH_TRAIN_DATA = list()
ALL_ZH_DEV_DATA = list()
ALL_ZH_TEST_DATA = list()

# only english
ALL_EN_TRAIN_DATA = list()
ALL_EN_DEV_DATA = list()
ALL_EN_TEST_DATA = list()

## 1. ALPACA

In [405]:
def read_alpaca():
    train_path = "./alpaca/train/train_52002.json"
    test_path = "./alpaca/test/test_252.json"
    
    def process(lines):
        new_lines = list()
        for line in tqdm(lines["instances"]):
            if "text" in line.keys():
                line["text"] = line["text"].replace("Defintion: More Information Needed \n ", "")
                line["text"] = line["text"].replace("Instruction: ", "")
                line["text"] = line["text"].replace("Input: ", USER_NAME + " ").replace("Output: ", OUR_NAME + " ")
                line["text"] = line["text"].strip().replace("\n\n", "\n") + " \n\n"
                line["text"] = line["text"].replace("\n {}".format(USER_NAME), "\n\n {}".format(USER_NAME))
            if "input" in line.keys():
                line["input"] = line["input"].replace("Defintion: More Information Needed \n ", "")
                line["input"] = line["input"].replace("Instruction: ", "")
                line["input"] = line["input"].replace("Input: ", USER_NAME + " ").replace("Output: ", OUR_NAME + " ")
                line["input"] = line["input"].strip().replace("\n\n", "\n") + " \n\n"
                line["input"] = line["input"].replace("\n {}".format(USER_NAME), "\n\n {}".format(USER_NAME))
            if "output" in line.keys():
                line["output"] = line["output"].replace("Defintion: More Information Needed \n ", "")
                line["output"] = line["output"].replace("Instruction: ", "")
                line["output"] = line["output"].replace("Input: ", USER_NAME + " ").replace("Output: ", OUR_NAME + " ")
                line["output"] = line["output"].strip().replace("\n\n", "\n") + " \n\n"
                line["output"] = line["output"].replace("\n {}".format(USER_NAME), "\n\n {}".format(USER_NAME))
            new_lines.append(line)
        return new_lines
    
    with open(train_path, "r", encoding="utf-8") as fr:
        train_lines = process(json.load(fr))
    with open(test_path, "r", encoding="utf-8") as fr:
        test_lines = process(json.load(fr))
    return train_lines, test_lines

alpaca_train_lines, alpaca_test_lines = read_alpaca()


100%|█████████████████████████████████████| 252/252 [00:00<00:00, 183246.29it/s]


In [406]:
ALL_EN_TRAIN_DATA.extend(alpaca_train_lines)
# ALL_TEST_DATA.extend(alpaca_test_lines)
# ALL_TRAIN_DATA.extend(hello_lines)

In [407]:
print("example: train_lines[0]=\n", alpaca_train_lines[55])
print()
print("train_num=", len(alpaca_train_lines))
print("test_num=", len(alpaca_train_lines))

example: train_lines[0]=
 {'text': '[Human]: What is the force on a 1 kg mass due to the gravitational force? \n [HugChat]: The force on a 1 kg mass due to the gravitational force is 9.8 N. \n\n'}

train_num= 52002
test_num= 52002


## 2. Belle_aplaca_cn

In [408]:
def read_alpaca_cn():
    train_path = "./alpaca_cn/Belle_open_source_1M.json"
    
    def process(lines):
        new_lines = list()
        for ei, line in enumerate(tqdm(lines)):
            line = json.loads(line.strip())
            input_text = line["instruction"]
            input_text = "{} {} \n ".format(USER_NAME, input_text.strip())
            output_text = "{} {}".format(OUR_NAME, line["output"].strip())
            text = input_text + output_text
            text = text.strip().replace("\n\n", "\n") + " \n\n"
            text = text.replace("\n {}".format(USER_NAME), "\n\n {}".format(USER_NAME))
            new_lines.append({
                "text": text
            })
#             if "[Human]: 你是谁" in text:
#                 print(ei)
        return new_lines
    
    with open(train_path, "r", encoding="utf-8") as fr:
        lines = fr.readlines()
        train_lines = process(lines)


    return train_lines

alpaca_cn_train_lines = read_alpaca_cn()


100%|███████████████████████████████| 917424/917424 [00:04<00:00, 205595.76it/s]


In [409]:
ALL_ZH_TRAIN_DATA.extend(alpaca_cn_train_lines)
# ALL_TRAIN_DATA.extend(hello_lines)

In [410]:
print("example: alpaca_train_lines=\n", alpaca_cn_train_lines[8589])
print()
print("len=", len(alpaca_cn_train_lines))

example: alpaca_train_lines=
 {'text': '[Human]: 编写一篇文章，主题为“如何在家庭中培养生态友好型生活方式”。 \n [HugChat]: 在如今越来越注重环境保护的时代，我们应该学会在日常生活中采取生态友好型的生活方式。以下是一些在家庭中培养生态友好型生活方式的建议：\n1.节约用水\n水是非常宝贵的资源，因此家庭中应该采取一些措施来节约用水。有些简单的方法包括：修复漏水的水龙头，减少冲厕所的次数以及减少洗澡的时间。此外，也可以安装节水装置来降低水的使用量。\n2.垃圾分类\n垃圾分类是一项非常重要的环保措施。在家庭中，我们应该把废纸、玻璃、塑料、金属等垃圾分离开，以便回收利用。同时，应该注意对一些有害垃圾进行正确的处理，例如电池、荧光灯等。\n3.减少使用化学品\n让我们的家庭环境更加安全和健康，我们应该尽可能减少使用化学物品。例如，我们可以选择使用环保清洁剂，而不是含有有害化学物质的清洁剂。同样，我们也应该尽可能选择无机的家具和装饰品。\n4.适当使用电能\n在家庭中使用电能也是我们应该注意的一点。要避免浪费电能，我们可以减少不必要的电器使用，比如打开电视却不看，使用更加节能的灯具等等。如果可能的话，还可以安装太阳能板等设备来更加环保地使用电能。\n总之，生态友好型的生活方式能够让我们的家庭环境更加清洁和健康。我们每个人都应该承担起责任，在日常生活中积极采取措施来保护我们的环境和地球。 \n\n'}

len= 917424


## 3. MultiTurn Chat

In [411]:
def read_multiturn_chat():
    train_path = "./multiturn_chat/multiturn_chat_0.8M.json"
    
    def process(lines):
        new_lines = list()
        for line in tqdm(lines):
            line = json.loads(line.strip())
            input_text = line["instruction"].strip()
            input_text = input_text.replace("\nHuman:", " \n " + USER_NAME).replace("Human:", USER_NAME)
            input_text = input_text.replace("\nAssistant:", " \n " + OUR_NAME)
            input_text = input_text.replace("\n [Human]:", "\n\n [Human]:")
            input_text += " "
            output_text = line["output"].strip() + " \n\n"
            text = input_text + output_text
            text = text.replace("[Human]: ", "[Human]:").replace("[Human]:", "[Human]: ")
            text = text.replace("[HugChat]: ", "[HugChat]:").replace("[HugChat]:", "[HugChat]: ")
            new_lines.append({
                "text": text
            })
        return new_lines
        
    with open(train_path, "r", encoding="utf-8") as fr:
        train_lines = fr.readlines()
        train_lines = process(train_lines)
    
    return train_lines

multiturn_chat_train_lines = read_multiturn_chat()


# print()
# print("train_num=", len(train_lines['instances']))

100%|████████████████████████████████| 831036/831036 [00:08<00:00, 98505.84it/s]


In [413]:
ALL_ZH_TRAIN_DATA.extend(multiturn_chat_train_lines)
# ALL_TRAIN_DATA.extend(hello_lines)

In [412]:
print("example: multiturn_chat_train_lines=\n", multiturn_chat_train_lines[243])
print()
print("len=", len(multiturn_chat_train_lines))

example: multiturn_chat_train_lines=
 {'text': '[Human]: 生成一段悬疑小说的开头。 \n [HugChat]: 深夜，雨声敲打着窗户，一个年轻女子拖着疲惫的身躯走进了一家小酒馆。酒馆里的人们都低着头，没有人与她搭话，仿佛她身上有不祥的气息。女子坐在角落里，注视着一名男子，拿起一杯酒匆匆离去。整个过程充满了神秘感，让人不寒而栗。 \n\n [Human]: 请给我一些创作写作灵感。 \n [HugChat]: 你可以试着写一篇关于一个人在旅行中遇到的故事，或者围绕着一段爱情故事展开你的想象。你也可以写一些关于科技或未来社会的故事，或者一些关于人类情感和人性的探讨。 \n\n [Human]: 回答一个封闭性问题，什么是机器学习？ \n [HugChat]: 机器学习是一种人工智能的应用程序，通过训练一个模型，使其可以根据已有的数据来完成某些任务，如图片分类、语音转文本等。其基本思想是利用数据来推断规律，并通过不断的优化来提高模型的精度和鲁棒性。机器学习在各行业具有广阔的应用前景，能够为人类带来巨大的效益。 \n\n [Human]: 开放性问题，你认为未来几年人工智能的发展趋势是什么？ \n [HugChat]: 未来几年人工智能会持续飞速发展，衍生出越来越多的新应用。在人工智能研究领域，深度学习、自然语言处理和计算机视觉等领域将得到更多的发展和突破。同时，人工智能在医疗、金融、交通等领域的应用将“智能化”程度进一步提高，为各行业带来巨大的效益。此外，人机协作、人工智能与物联网的结合也将是未来发展的重要趋势。 \n\n'}

len= 831036


## 4. GPT4 data clean❌

In [68]:
def read_gpt4_clean():
    train_path = "./gpt4all_data_clean/data.jsonl"
    
    def process(lines):
        new_lines = list()
        for line in tqdm(lines):
            line = json.loads(line.strip())
            input_text = line["prompt"][3: -4]
            input_text = "{} {} \n ".format(USER_NAME, input_text)
            output_text = line["response"].strip()
            output_text = "{} {} \n\n".format(OUR_NAME, output_text)
            text = input_text + output_text
            new_lines.append({
                "text": text
            })
        return new_lines
        
    with open(train_path, "r", encoding="utf-8") as fr:
        train_lines = fr.readlines()
        train_lines = process(train_lines)
    
    return train_lines

gpt4_clean = read_gpt4_clean()


# print()
# print("train_num=", len(train_lines['instances']))

100%|███████████████████████████████| 437605/437605 [00:03<00:00, 128052.02it/s]


In [315]:
ALL_ZH_TRAIN_DATA.extend(gpt4_clean)
# ALL_TRAIN_DATA.extend(hello_lines)

NameError: name 'gpt4_clean' is not defined

In [72]:
print("example: gpt4_clean=\n", gpt4_clean[1032])
print()
print("len=", len(gpt4_clean))

example: gpt4_clean=
 {'text': 'Human: I do not know why but I have problems with this code. The banner is displayed on every page although it has specified the attribute $(location).attr(\'href\') you can help me?:</p>\n\n<pre><code>&lt;div id="bottombar"&gt;\n&lt;div class="bottom-content"&gt;\n&lt;a href="http://www.cliente.org/" target="_blank"&gt;&lt;img src="&lt;?php bloginfo(\'stylesheet_directory\'); ?&gt;/images/banner/bottom-logo.png" alt="player-logo" /&gt;&lt;/a&gt;\n&lt;a href="http://www.cliente.org/" target="_blank"&gt;&lt;img src="&lt;?php bloginfo(\'stylesheet_directory\'); ?&gt;/images/banner/bottom-txt.png" alt="player-slogan" /&gt;&lt;/a&gt;\n&lt;img src="&lt;?php bloginfo(\'stylesheet_directory\'); ?&gt;/images/banner/bottom-download.png" alt="player-download" /&gt;\n&lt;/div&gt;\n&lt;div id="bottombarClose"&gt;&lt;p&gt;Chiudi&lt;/p&gt;&lt;/div&gt;\n&lt;script type="text/javascript""&gt;\n$(document).ready(function() {\nvar currentUrl = $(location).attr(\'href\');\

## Math

In [414]:
def read_math():
    train_path = "./math/school_math_0.25M.json"
    
    def process(lines):
        new_lines = list()
        for line in tqdm(lines):
            line = json.loads(line.strip())
            input_text = line["instruction"]
            input_text = "{} {} \n ".format(USER_NAME, input_text)
            input_text = input_text.replace("\n\n", "\n")
            output_text = line["output"].strip().replace("\n\n", "\n")
            output_text = "{} {} \n\n".format(OUR_NAME, output_text)
            text = input_text + output_text
            new_lines.append({
                "text": text
            })
        return new_lines
        
    with open(train_path, "r", encoding="utf-8") as fr:
        train_lines = fr.readlines()
        train_lines = process(train_lines)
    
    return train_lines

math_data = read_math()


# print()
# print("train_num=", len(train_lines['instances']))

100%|███████████████████████████████| 248481/248481 [00:01<00:00, 227410.36it/s]


In [415]:
ALL_ZH_TRAIN_DATA.extend(math_data)
# ALL_TRAIN_DATA.extend(hello_lines)

In [474]:
print("example: math_data=\n", math_data[12])
print()
print("len=", len(math_data))

example: math_data=
 {'text': '[Human]: 小明家里有3个苹果，他给了小红1个，小明还剩几个苹果？ \n [HugChat]: - 小明前有苹果：3 个 \n- 小明给了小红一个：-1 个 \n- 所以，小明还剩下的苹果是：2 个 \n\n'}

len= 248481


## MedMCQA

In [417]:
def read_medmcqa():
    train_path = "./MedMCQA/train/train_182822.json"
    test_path = "./MedMCQA/validation/valid_4183.json"
    
    def process(lines):
        new_lines = list()
        for line in tqdm(lines["instances"]):
            if "text" in line.keys():
                line["text"] = line["text"].replace("Defintion: More Information Needed \n ", "")
                line["text"] = line["text"].replace("Instruction: ", "")
                line["text"] = line["text"].replace("Input: ", USER_NAME + " ").replace("Output: ", OUR_NAME + " ")
                line["text"] = line["text"].strip().replace("\n\n", "\n") + " \n\n"
            if "input" in line.keys():
                line["input"] = line["input"].replace("Defintion: More Information Needed \n ", "")
                line["input"] = line["input"].replace("Instruction: ", "")
                line["input"] = line["input"].replace("Input: ", USER_NAME + " ").replace("Output: ", OUR_NAME + " ")
                line["input"] = line["input"].strip().replace("\n\n", "\n") + " \n\n"
            if "output" in line.keys():
                line["output"] = line["output"].replace("Defintion: More Information Needed \n ", "")
                line["output"] = line["output"].replace("Instruction: ", "")
                line["output"] = line["output"].replace("Input: ", USER_NAME + " ").replace("Output: ", OUR_NAME + " ")
                line["output"] = line["output"].strip().replace("\n\n", "\n") + " \n\n"
            new_lines.append(line)
        return new_lines
    
    with open(train_path, "r", encoding="utf-8") as fr:
        train_lines = process(json.load(fr))
    with open(test_path, "r", encoding="utf-8") as fr:
        test_lines = process(json.load(fr))
    return train_lines, test_lines

medmcqa_train_lines, medmcqa_test_lines = read_medmcqa()


100%|███████████████████████████████████| 4183/4183 [00:00<00:00, 436491.45it/s]


In [418]:
ALL_EN_TRAIN_DATA.extend(medmcqa_train_lines)
# ALL_TEST_DATA.extend(medmcqa_test_lines)
# ALL_TRAIN_DATA.extend(hello_lines)

In [473]:
print("example: medmcqa_train_lines[0]=\n", medmcqa_train_lines[1134])
print()
print("train_num=", len(medmcqa_train_lines))
print("test_num=", len(medmcqa_test_lines))

example: medmcqa_train_lines[0]=
 {'text': '[Human]: Question: True regarding the rubens valve is (A) Is a non-rebreathing valve (B) Will not allow spontaneous respiration (C) Is a pressure reducing valve (D) May jam in expiratory position. \n [HugChat]: A. \n\n'}

train_num= 182822
test_num= 4183


## MedQA-USMLE

In [420]:
def read_medqa():
    train_path = "./MedQA-USMLE/train/train_10178.json"
    test_path = "./MedQA-USMLE/validation/valid_1273.json"
    
    def process(lines):
        new_lines = list()
        for line in tqdm(lines["instances"]):
            if "text" in line.keys():
                line["text"] = line["text"].replace("Defintion: More Information Needed \n ", "")
                line["text"] = line["text"].replace("Instruction: ", "")
                line["text"] = line["text"].replace("Input: ", USER_NAME + " ").replace("Output: ", OUR_NAME + " ")
                line["text"] = line["text"].strip().replace("\n\n", "\n") + " \n\n"
            if "input" in line.keys():
                line["input"] = line["input"].replace("Defintion: More Information Needed \n ", "")
                line["input"] = line["input"].replace("Instruction: ", "")
                line["input"] = line["input"].replace("Input: ", USER_NAME + " ").replace("Output: ", OUR_NAME + " ")
                line["input"] = line["input"].strip().replace("\n\n", "\n") + " \n\n"
            if "output" in line.keys():
                line["output"] = line["output"].replace("Defintion: More Information Needed \n ", "")
                line["output"] = line["output"].replace("Instruction: ", "")
                line["output"] = line["output"].replace("Input: ", USER_NAME + " ").replace("Output: ", OUR_NAME + " ")
                line["output"] = line["output"].strip().replace("\n\n", "\n") + " \n\n"
            new_lines.append(line)
        return new_lines
    
    with open(train_path, "r", encoding="utf-8") as fr:
        train_lines = process(json.load(fr))
    with open(test_path, "r", encoding="utf-8") as fr:
        test_lines = process(json.load(fr))
    return train_lines, test_lines

medqa_train_lines, medqa_test_lines = read_medqa()


100%|███████████████████████████████████| 1273/1273 [00:00<00:00, 191966.24it/s]


In [421]:
ALL_EN_TRAIN_DATA.extend(medqa_train_lines)
# ALL_TEST_DATA.extend(medqa_test_lines)
# ALL_TRAIN_DATA.extend(hello_lines)

In [422]:
print("example: medqa_train_lines[0]=\n", medqa_train_lines[0])
print()
print("train_num=", len(medqa_train_lines))
print("test_num=", len(medqa_train_lines))

example: medqa_train_lines[0]=
 {'text': '[Human]: Question: A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus. Which of the following is the best treatment for this patient? (A) Ampicillin (B) Ceftriaxone (C) Doxycycline (D) Nitrofurantoin. \n [HugChat]: D \n\n'}

train_num= 10178
test_num= 10178


## PubMedQA

In [423]:
def read_pubmedqa():
    train_path = "./PubMedQA/train/train_211269.json"
    test_path = "./PubMedQA/test/test_1000.json"
    
    def process(lines):
        new_lines = list()
        for line in tqdm(lines["instances"]):
            if "text" in line.keys():
                line["text"] = line["text"].replace("Defintion: More Information Needed \n ", "")
                line["text"] = line["text"].replace("Instruction: ", "")
                line["text"] = line["text"].replace("Input: ", USER_NAME + " ").replace("Output: ", OUR_NAME + " ")
            if "input" in line.keys():
                line["input"] = line["input"].replace("Defintion: More Information Needed \n ", "")
                line["input"] = line["input"].replace("Instruction: ", "")
                line["input"] = line["input"].replace("Input: ", USER_NAME + " ").replace("Output: ", OUR_NAME + " ")
            if "output" in line.keys():
                line["output"] = line["output"].replace("Defintion: More Information Needed \n ", "")
                line["output"] = line["output"].replace("Instruction: ", "")
                line["output"] = line["output"].replace("Input: ", USER_NAME + " ").replace("Output: ", OUR_NAME + " ")
            new_lines.append(line)
        return new_lines
    
    with open(train_path, "r", encoding="utf-8") as fr:
        train_lines = process(json.load(fr))
    with open(test_path, "r", encoding="utf-8") as fr:
        test_lines = process(json.load(fr))
    return train_lines, test_lines

pubmedqa_train_lines, pubmedqa_test_lines = read_pubmedqa()


100%|███████████████████████████████████| 1000/1000 [00:00<00:00, 203547.70it/s]


In [424]:
ALL_EN_TRAIN_DATA.extend(pubmedqa_train_lines)
# ALL_TEST_DATA.extend(pubmedqa_test_lines)
# ALL_TRAIN_DATA.extend(hello_lines)

In [472]:
print("example: pubmedqa_train_lines[0]=\n", pubmedqa_train_lines[1233])
print()
print("train_num=", len(pubmedqa_train_lines))
print("test_num=", len(pubmedqa_train_lines))

example: pubmedqa_train_lines[0]=
 {'text': '[Human]: Context: Coronary artery calcification (CAC) is a prominent feature of atherosclerosis and is associated with cardiovascular events. In vitro studies have suggested that osteoprotegerin (OPG) and osteocalcin (OC) exert anticalcification potential in the vessel wall. The objective of this study was to investigate the association of CAC and serum bone biomarkers in persons with type 2 diabetes. We examined 50 individuals with type 2 diabetes. CAC imaging was performed by multidetector computed tomography. CAC scores ≥10, expressed in Agatston units, were considered abnormal. OC, undercarboxylated OC (ucOC), and OPG levels were determined by enzyme-linked immunosorbent assay. Abnormal CAC scores were found for 64% of the study cohort. OPG levels were significantly elevated (5.5 ± 2.0 pmol/L vs. 4.2 ± 1.7 pmol/L; P = .026) for those with abnormal CAC scores. No univariate differences were found for OC or ucOC. Logistic regression analys

## OID

In [426]:
import os
from random import shuffle
file_list = os.listdir("./OID")
print(file_list)

['unified_soda_dialog.jsonl', 'unified_abstract_infill.jsonl', 'unified_unnatural_instructions.jsonl', 'unified_poetry_instructions.jsonl', 'unified_flan.jsonl', '.DS_Store', 'unified_multi_news.jsonl', 'unified_grade_school_math_instructions.jsonl', 'unified_chip2.jsonl', 'unified_joke_explanations.jsonl', 'unified_openai_summarize_tldr.jsonl', 'unified_merged_code_xp3.jsonl', 'unified_essays.jsonl', 'unified_conv_finqa.jsonl', 'unified_hc3_human.jsonl', 'unified_cuad.jsonl', 'unified_squad_v2.jsonl']


In [427]:
ignore_files = [
    '.DS_Store',
    'unified_poetry_instructions.jsonl', 'unified_multi_news.jsonl', 
    'unified_merged_code_xp3.jsonl', 'unified_cuad.jsonl', "unified_flan.jsonl", "unified_openai_summarize_tldr.jsonl"
]

In [428]:
language = {
    "unified_soda_dialog.jsonl": "en",
    "unified_abstract_infill.jsonl": "en",
    "unified_unnatural_instructions.jsonl": "en",
    "unified_poetry_instructions.jsonl": "en",
    "unified_flan.jsonl": "en",
    "unified_multi_news.jsonl": "",
    "unified_grade_school_math_instructions.jsonl": "",
    "unified_chip2.jsonl": "",
    "unified_joke_explanations.jsonl": "",
    "unified_openai_summarize_tldr.jsonl": "",
    "unified_merged_code_xp3.jsonl": "",
    "unified_essays.jsonl": "",
    "unified_conv_finqa.jsonl": "",
    "unified_hc3_human.jsonl": "",
    "unified_cuad.jsonl": "",
    "unified_squad_v2.jsonl": "",
}

In [429]:
def read_oid(train_path):
    
    def process(lines):
        new_lines = list()
        for line in tqdm(lines[:200000]):
            line = json.loads(line.strip())
            input_text = line["text"]
            input_text = input_text.replace("\n<human>:", " \n " + USER_NAME).replace("<human>:", USER_NAME)
            input_text = input_text.replace("\n<bot>:" , " \n " + OUR_NAME)
            input_text = input_text.replace("\nRosey: ", " \n " + OUR_NAME + " ")
            input_text = input_text.replace("\n\n", "\n")
            input_text = input_text.replace("\n {}".format(USER_NAME), "\n\n {}".format(USER_NAME))
            input_text = input_text.strip()  + " \n\n"
            new_lines.append({
                "text": input_text
            })
        return new_lines
        
    with open(train_path, "r", encoding="utf-8") as fr:
        train_lines = fr.readlines()
        train_lines = process(train_lines)
    
    return train_lines

# print()
# print("train_num=", len(train_lines['instances']))

In [378]:
f_n = file_list[14]
print(f_n)
train_lines = read_oid("./OID/{}".format(f_n))

unified_hc3_human.jsonl


100%|█████████████████████████████████| 58546/58546 [00:00<00:00, 123622.97it/s]


In [122]:
shuffle(train_lines)

In [470]:
train_lines[223]

{'text': '[Human]: Are there any ethical considerations when building a chatbot? \n [HugChat]: Absolutely! It is important to think carefully about ethical considerations when creating a chatbot. For example, chatbots should not be used to manipulate people or spread false information. All chatbot interactions should be respectful, transparent, and honest. Additionally, chatbots should have clear terms of use and privacy policies. \n\n [Human]: What are some of the tasks that a chatbot can do? \n [HugChat]: Chatbots can do many tasks, such as answering questions, providing definitions and explanations, translating text from one language to another, summarizing text, generating text, writing stories, analyzing sentiment, providing recommendations, developing algorithms and creating code, and any other language-based tasks. \n\n'}

In [430]:
oid_data = list()
for file_name in file_list:
    if file_name in ignore_files:
        continue
    data = read_oid("./OID/{}".format(file_name))
    oid_data.extend(data)

100%|██████████████████████████████████| 19586/19586 [00:00<00:00, 61175.68it/s]


In [431]:
ALL_EN_TRAIN_DATA.extend(oid_data)
# ALL_TRAIN_DATA.extend(hello_lines)

In [468]:
print(oid_data[2324])

{'text': '[Human]: Can you give me a short story description for this dialog?\n  Fatou: Hey, Dad. I need to talk to you about something.\n  Dad: What\'s up?\n  Fatou: I\'m pregnant.\n  Dad: Oh, Fatou. I\'m so sorry.\n  Fatou: No, no, it\'s okay! I\'m happy about it, really. It\'s just that, well, I\'m not sure what to do next.\n  Dad: Have you told the father yet?\n  Fatou: Yeah, he\'s really supportive and wants to help out however he can.\n  Dad: That\'s good. You\'re lucky to have him be so supportive. But what about you? How are you feeling about all of this?\n  Fatou: I\'m scared but mostly excited. This is something I\'ve always wanted but never thought would happen for me so it feels like a dream come true in a lot of ways. But then there are all these practical considerations and I don\'t know what to do or where to start with any of it.\n  Dad: Well, first things first, we need to figure out how you\'re going to afford everything on your own since insurance probably won\'t cov

In [32]:
print("example: oid_data[0]=\n", oid_data[0])
print()
print("train_num=", len(oid_data))
print("test_num=", len(oid_data))

example: oid_data[0]=
 {'text': 'Human: Can you write short story and dialog based on the theme:\n  being interested \n HugChat: Sure, a short story and dialog based on the theme "being interested" could be:\n  Now that she had a specific goal in mind, Jacquelin started looking for a new car. She began by comparing different models online, and then went to test drive them at the dealership. After doing some research and taking some test drives, Jacquelin decided on the perfect car for her.\n  Jacquelin: I\'m looking for a new car.\n  Salesman: What kind of car are you interested in?\n  Jacquelin: I\'m not really sure. I just know that I want something different from what I have now.\n  Salesman: Well, let\'s take a look at our inventory and see if anything catches your eye.\n  Jacquelin: Okay.\n  Salesman: We have a lot of great cars here. What is your budget?\n  Jacquelin: $15,000.\n  Salesman: We have a few options within your budget. This Honda Civic is a great choice. It\'s very re

In [33]:
print("len(ALL_TRAIN_DATA)=", len(ALL_TRAIN_DATA))
print("len(ALL_TEST_DATA)=", len(ALL_TEST_DATA))

len(ALL_TRAIN_DATA)= 5465685
len(ALL_TEST_DATA)= 6708


# MOSS

In [432]:
import os
from random import shuffle
moss_file_list = [i for i in os.listdir("./fudan/") if i != ".DS_Store" ]
print(moss_file_list)

['en_honesty.json', 'zh_honesty.json', 'en_helpfulness.json', 'en_harmlessness.json', 'zh_helpfulness.json']


In [433]:
import json
def read_moss(train_path):
    
    def process(lines):
        new_lines = list()
        for ei, line in enumerate(tqdm(lines)):
#             if "en_honesty.json" in train_path and ei in [6096, 6384]:
#                 continue
            input_text = line["plain_text"].replace("\n\n", "\n")
            input_text = input_text.replace("<eoh>", " \n").replace("<eoa>", " \n\n")
            input_text = input_text.replace("[MOSS]:", OUR_NAME)
            
            input_text = input_text.replace("fudannlp@fudan.edu.cn", "hugailab@gmail.com or lygwjn@gmail.com")
            input_text = input_text.replace("https://fudannlp.org/", "https://github.com/HugAILab/")
            input_text = input_text.replace("FudanNLP Lab", "HugAI Lab").replace("Fudan University", "East China Normal University")
            input_text = input_text.replace("February 7, 2023", "May 2, 2023")
            input_text = input_text.replace("moss@fudan.edu.cn", "hugailab@gmail.com").replace("fudan.edu.cn/moss", "https://github.com/HugAILab/HugNLP")
            input_text = input_text.replace("support@fudannlp.com.", "hugailab@gmail.com or lygwjn@gmail.com").replace("http://fudannlp.com/moss", "https://github.com/HugAILab/HugNLP")
            input_text = input_text.replace("support@fudanNLP.com", "hugailab@gmail.com or lygwjn@gmail.com").replace("http://www.fudannlp.org/.", "https://wjn1996.github.io/.")
            input_text = input_text.replace("MOSS", "HugChat").replace("2023年2月7日", "2023年5月2日")
            input_text = input_text.replace("160亿", "100亿左右")
            input_text = input_text.replace("复旦大学自然语言处理实验室", "华东师范大学数据学院").replace("上海人工智能实验室", "HugAI Lab")
            input_text = input_text.replace("复旦大学", "华东师范大学").replace("复旦", "华师")
            
            input_text_list = input_text.split("[Human]")
            new_input_text_list = list()
            for tt in input_text_list:
                if "fudan" in tt or "shlab" in tt:
                    continue
                new_input_text_list.append(tt)
            input_text = "[Human]".join(new_input_text_list)
            if "fudan" in input_text:
                print(ei)
            if "zhangyue yin" in input_text:
                print(ei)
            if "复旦" in input_text:
                print(ei)
            new_lines.append({
                "text": input_text
            })
        return new_lines
        
    with open(train_path, "r", encoding="utf-8") as fr:
#         train_lines = fr.read().decode(encoding='gbk')
#         train_lines = fr.readlines()
        train_lines = json.load(fr)
#         print(train_lines[0])
        train_lines = process(train_lines)
    
    return train_lines



In [460]:
f_n = moss_file_list[0]
print(f_n)
train_lines = read_moss("./fudan/{}".format(f_n))

en_honesty.json


100%|████████████████████████████████| 112580/112580 [00:05<00:00, 20838.37it/s]


In [465]:
print(train_lines[934])

{'text': "[Human]: How can I measure the effectiveness of my chatbot's responses? \n [HugChat]: There are several metrics that can be used to measure the effectiveness of a chatbot's responses. These include accuracy, speed, relevance, and usefulness. Accuracy measures how often the chatbot correctly responds to a query or request, while speed measures how quickly the chatbot responds. Relevance measures how closely the chatbot's responses match the user's intent, and usefulness measures the practicality of the response. \n\n [Human]: Can you provide some examples of accuracy metrics? \n [HugChat]: Sure! Accuracy metrics can be measured by comparing the chatbot's response with a manually-created response. For example, one metric you could use is the BLEU score, which measures the similarity between the two outputs. Other metrics include precision, recall, and F1 score. \n\n [Human]: How can I improve accuracy? \n [HugChat]: Improving accuracy can be done by training the chatbot on more

In [434]:
moss_data = list()
for file_name in moss_file_list:
    data = read_moss("./fudan/{}".format(file_name))
    moss_data.extend(data)

100%|████████████████████████████████| 447750/447750 [00:05<00:00, 81726.80it/s]


In [475]:
print(len(moss_data))

1161137


In [435]:
ALL_ZH_TRAIN_DATA.extend(moss_data)

# MERGE ALL DATA

In [440]:
shuffle(ALL_ZH_TRAIN_DATA)
print("len(zh)=", len(ALL_ZH_TRAIN_DATA))
shuffle(ALL_EN_TRAIN_DATA)
print("len(en)=", len(ALL_EN_TRAIN_DATA))
ALL_TRAIN = ALL_ZH_TRAIN_DATA + ALL_EN_TRAIN_DATA
shuffle(ALL_TRAIN)
print("len(all)=", len(ALL_TRAIN))

len(zh)= 3158078
len(en)= 1354726
len(all)= 4512804


In [443]:
print(ALL_ZH_TRAIN_DATA[43])

{'text': '[Human]: 给出三个单词，使用它们创建一个句子并输出结果。 \n单词：猫，鱼，看 \n [HugChat]: 猫喜欢看鱼。 \n\n'}


In [441]:
with open("instruction_en_corpora.json", "w", encoding="utf-8") as fw:
    for line in tqdm(ALL_EN_TRAIN_DATA):
        json_d = {}
        try:
            json_d["prefix"] = instruct
            json_d["text"] = line["text"]
        except:
            print(type(line))
            break
        fw.write(json.dumps(json_d, ensure_ascii=False) + "\n")

100%|██████████████████████████████| 1354726/1354726 [00:14<00:00, 90345.16it/s]


In [442]:
with open("instruction_zh_corpora.json", "w", encoding="utf-8") as fw:
    for line in tqdm(ALL_ZH_TRAIN_DATA):
        json_d = {}
        try:
            json_d["prefix"] = instruct
            json_d["text"] = line["text"]
        except:
            print(type(line))
            break
        fw.write(json.dumps(json_d, ensure_ascii=False) + "\n")

100%|██████████████████████████████| 3158078/3158078 [00:38<00:00, 82255.05it/s]


In [444]:
with open("instruction_corpora.json", "w", encoding="utf-8") as fw:
    for line in tqdm(ALL_TRAIN):
        json_d = {}
        try:
            json_d["prefix"] = instruct
            json_d["text"] = line["text"]
        except:
            print(type(line))
            break
        fw.write(json.dumps(json_d, ensure_ascii=False) + "\n")

100%|██████████████████████████████| 4512804/4512804 [01:07<00:00, 67204.88it/s]


In [35]:
with open("instruction_corpora_small.json", "w", encoding="utf-8") as fw:
    for line in tqdm(ALL_TRAIN_DATA[:10000]):
        json_d = {}
        try:
            json_d["prefix"] = instruct
            json_d["text"] = line["text"]
        except:
            print(type(line))
            break
        fw.write(json.dumps(json_d, ensure_ascii=False) + "\n")

100%|█████████████████████████████████| 10000/10000 [00:00<00:00, 116290.08it/s]


In [459]:
max_len = 0
max_str = ""
small = 0
for line in tqdm(ALL_EN_TRAIN_DATA):
    if len(line["text"].split(" ")) < 512:
        small += 1
    if len(line["text"].split(" ")) > max_len:
        max_str = line["text"]
    max_len = max(max_len, len(line["text"].split(" ")))
print(max_len)
# print(max_str)
print(small)

100%|██████████████████████████████| 1354726/1354726 [00:46<00:00, 29407.82it/s]

54477
1299211
